In [1]:
# installing requirements from txt file
#pip install -r requirements.txt

In [2]:
# importing necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import yfinance as yf
from datetime import datetime

# **Step 1: Data retrieval and indicators calculation**

The idea is to consider a portfolio made only of SPY ETF, as it has been seen that holding an ETF which replicates the Standard and Poor 500 can be one of the best investments you can make.

In [3]:
# downloading monthly prices of the SPY ETF, as VIX data will be monthly and therefore we keep returns as monthly
spy_prices = yf.download('SPY', start = '2000-01-01', end = '2024-12-31', interval = '1mo')
spy_prices = spy_prices['Adj Close']
spy_prices

[*********************100%%**********************]  1 of 1 completed


Date
2000-01-01     88.948593
2000-02-01     87.594223
2000-03-01     95.839806
2000-04-01     92.708878
2000-05-01     91.251236
                 ...    
2024-08-01    560.071289
2024-09-01    570.086792
2024-10-01    566.732605
2024-11-01    600.528809
2024-12-01    584.114075
Name: Adj Close, Length: 300, dtype: float64

In [4]:
spy_rets = spy_prices.pct_change().dropna()
spy_rets

Date
2000-02-01   -0.015226
2000-03-01    0.094134
2000-04-01   -0.032668
2000-05-01   -0.015723
2000-06-01    0.017286
                ...   
2024-08-01    0.023365
2024-09-01    0.017883
2024-10-01   -0.005884
2024-11-01    0.059633
2024-12-01   -0.027334
Name: Adj Close, Length: 299, dtype: float64

Next, we upload VIX future (UX1 index) term structure data downloaded from Bloomberg as of 1/17/2025:

In [14]:
vix_data = pd.read_excel('VIX_term_structure_20250117.xlsx', header = 0)
vix_data = vix_data.drop(vix_data.index[0]) # removing first unnecessary row
vix_data

,Tenor,Ticker,Period,Last Price,Days to expiration
1,Spot,VIX Index,Spot,15.9700,0.0
2,1M,UXF5 Index,01/2025,16.1792,30.0
3,1M,UXG5 Index,02/2025,17.2382,60.0
4,2M,UXH5 Index,03/2025,17.8351,90.0
5,3M,UXJ5 Index,04/2025,18.1962,120.0
6,4M,UXK5 Index,05/2025,18.4005,150.0
7,5M,UXM5 Index,06/2025,18.5484,180.0
8,6M,UXN5 Index,07/2025,18.8250,210.0
9,7M,UXQ5 Index,08/2025,18.8000,240.0
10,8M,UXU5 Index,09/2025,19.1000,270.0


In [47]:
# creating functions for the three indicators which will compose the innovative part of our approach

def rolling_std(series, time_interval): # defining a function for volatility, which we consider as rolling standard deviation
    return series.rolling(window = time_interval).std()

def rolling_correlation(series1, series2, time_interval): # defining a function for the rolling correlation
    return series1.rolling(window = time_interval).corr(series2)

def ROC(series): # defining a function for the Rate Of Change
    return series.pct_change()

Now we are going to define a function that will create a new term structure with newly calculated prices.
The prices will be calculated via linear interpolation with a targeted maturity, so that the new term structure is characterized by constant maturity.

In [26]:
def constant_maturity_term_structure(data, target_maturity):
    """
    This function computes the linear interpolation of VIX futures prices 
    for generating a constant maturity term structure.
    
    It takes the VIX dataframe with prices and days to expiration,
    together with a target maturity, expressed in days, as inputs,
    and will return the interpolated prices of the VIX futures.
    """
    constant_prices = [] # pre-allocating memory for the prices calculated via constant maturity term structure
    
    for target in target_maturity: # we will be looping over all maturities included in the table
        before = data[data["Days to expiration"] <= target] # looking for the nearest earlier contract for interpolation
        after = data[data["Days to expiration"] > target] # looking for the nearest later contract for interpolation

        # for different maturities, there will be cases where we won't have the directly close value, therefore
        if before.empty:  # if the price on the earlier place is empty
            price = after.iloc[0]["Last Price"]  # we will take the earliest available price
        elif after.empty:  # if the price on the later place is empty
            price = before.iloc[-1]["Last Price"]  # use the latest available
        else:
            # otherwise
            before = before.iloc[-1] # the earlier price is normally identified
            after = after.iloc[0] # the later price is normally identified
            price = before["Last Price"] + (
                (after["Last Price"] - before["Last Price"]) /
                (after["Days to expiration"] - before["Days to expiration"]) *
                (target - before["Days to expiration"])
            ) # here we calculate linear interpolation through a commonly accepted and recognized formula
        
        constant_prices.append(price) # for each iteration of the loop, we append the newly calculated price to the originally created variable
    
    return constant_prices # the function will return the newly calculated prices

Running the function for our data:

In [44]:
target_maturity = [30, 60, 90, 120, 150, 180, 210, 240, 270]  # here we define a variable for the target maturities and it will be all of them

constant_maturity = constant_maturity_term_structure(vix_data, target_days) 

# Add the constant maturity prices to a DataFrame
constant_maturity_df = pd.DataFrame({
    "Constant Maturity (Days)": target_maturity,
    "Price": constant_maturity
})

print("Constant Maturity Term Structure:") # printing the new term structure, made of prices at constant maturity
print(constant_maturity_df)

Constant Maturity Term Structure:
   Constant Maturity (Days)    Price
0                        30  16.1792
1                        60  17.2382
2                        90  17.8351
3                       120  18.1962
4                       150  18.4005
5                       180  18.5484
6                       210  18.8250
7                       240  18.8000
8                       270  19.1000


Calculating the slope of the constant maturity VIX futures term structure in a normalized way, meaning with the difference in prices at the numerator and the difference in days at the denominator:

In [48]:
vix_slope = constant_maturity_df["Price"].diff() / constant_maturity_df["Constant Maturity (Days)"].diff() # computing the slope of the term structure
vix_slope = vix_slope.dropna()
vix_slope

1    0.035300
2    0.019897
3    0.012037
4    0.006810
5    0.004930
6    0.009220
7   -0.000833
8    0.010000
dtype: float64

Using the previously defined three functions, we are computing the innovative indicators that we are going to use as a double check after the momentum transformer:

In [52]:
vol_indicator = rolling_std(spy_rets, time_interval = 10).dropna() # calculating volatility indicator on the returns of SPY
correlation_indicator = rolling_correlation(spy_rets, vix_slope, time_interval = 30).dropna()
roc_indicator = ROC(vix_slope).dropna() # calculating rate of change of the VIX futures constant maturity term structure slope

print(vol_indicator)

Date
2000-11-01    0.051522
2000-12-01    0.051391
2001-01-01    0.043089
2001-02-01    0.051015
2001-03-01    0.052970
                ...   
2024-08-01    0.032780
2024-09-01    0.025355
2024-10-01    0.026244
2024-11-01    0.029119
2024-12-01    0.031262
Name: Adj Close, Length: 290, dtype: float64
Series([], dtype: float64)
2    -0.436355
3    -0.395041
4    -0.434229
5    -0.276065
6     0.870183
7    -1.090383
8   -13.000000
dtype: float64


In [54]:
print(correlation_indicator)

Series([], dtype: float64)


In [ ]:
print(roc_indicator)